In [9]:
#Imports
import math
import requests
import json
import csv
import pandas as pd
import requests
from requests.exceptions import HTTPError
import re
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.safari.service import Service
import time


In [10]:
data = pd.read_csv("ScopusSearch.csv")
data1 = pd.read_csv("AbstractRetrieval.csv")
mergeData = pd.merge(data, data1, on='eid', how='left')
mergeData

In [3]:
mergeData.columns

In [4]:
springer_filter = mergeData[
    mergeData['publisher'].str.contains('Springer', case=False, na=False) |
    mergeData['publisher'].str.contains('Institute of Electrical and Electronics Engineers Inc', case=False, na=False)
]
len(springer_filter)

In [5]:
springer_filter['publisher'].value_counts()

In [9]:
springer_filter

In [6]:
springer_filter['subtypeDescription'].unique()

In [15]:
springer_filter = mergeData[mergeData['subtypeDescription'].isin(['Article', 'Conference Paper'])]
springer_filter['subtypeDescription'].unique()


In [16]:
key = springer_filter[springer_filter['authkeywords'].str.contains('Artificial intelligence', case=False, na=False)]
len(key)

In [6]:
key['authkeywords']

In [8]:
key.to_csv("AIfilter.csv", index=False)

## Se inicia la sesion 

In [10]:
service = Service('/usr/bin/safaridriver')  # SafariDriver está ubicado en /usr/bin/safaridriver
driver = webdriver.Safari(service=service)

In [11]:
#Stringer#
Stringer = 'https://doi.org/10.1007/s10845-020-01712-9'
driver.get(Stringer)

time.sleep(2)
cokies = driver.find_element(By.CSS_SELECTOR,".cc-button--secondary")
cokies.click() #cokies


button = driver.find_element(By.CSS_SELECTOR,".app-article-access .u-button--primary, .app-article-access .u-button--primary:visited")
button.click()


time.sleep(5)
cokies = driver.find_element(By.CSS_SELECTOR,".cc-button--secondary")
cokies.click() ## cokies

button = driver.find_element(By.CSS_SELECTOR,"*[data-test=search]")
button.click()


button.send_keys('UTPl')
time.sleep(5)
button = driver.find_element(By.CSS_SELECTOR,".c-results-container__result")
button.click()


time.sleep(10)
us = driver.find_element(By.ID,"userNameInput")
us.click()
us.send_keys('trgaona1@utpl.edu.ec')

us = driver.find_element(By.ID,"passwordInput")
us.click()
us.send_keys('j5hfrWRM')


us = driver.find_element(By.ID,"submitButton")
us.click()

#IEEE#

IEEE = 'https://doi.org/10.1109/ISDFS60797.2024.10527283'
driver.get(IEEE)
driver.get(IEEE)
time.sleep(10)
button = driver.find_element(By.CSS_SELECTOR,".institution-container .inst-sign-in")
button.click()
time.sleep(5)

button = driver.find_element(By.CLASS_NAME,"seamless-access-btn")
button.click()

time.sleep(5)
button = driver.find_element(By.CLASS_NAME,"inst-typeahead-input")
button.click()
button.send_keys('loja')

time.sleep(5)
button = driver.find_element(By.ID,"Universidad TÃ©cnica Particular de Loja")
button.click()

"""
time.sleep(5)
us = driver.find_element(By.ID,"userNameInput")
us.click()
us.send_keys('trgaona1@utpl.edu.ec')

us = driver.find_element(By.ID,"passwordInput")
us.click()
us.send_keys('j5hfrWRM')

us = driver.find_element(By.ID,"submitButton")
us.click()
"""

### Extracción de PDFs

In [22]:
a = []
z = []
i = 1
x = 1
for c in key.itertuples():
    str = "https://doi.org/"+c.doi
    try:
        time.sleep(5)
        driver.get(str)
        page_source = driver.page_source
        soup = BeautifulSoup(page_source, 'html.parser')
        pdf_link_tag = soup.find('a', class_='u-button u-button--full-width u-button--primary u-justify-content-space-between c-pdf-download__link')
        pdf_url = "https://link.springer.com" + pdf_link_tag['href']
        if pdf_url.endswith(".pdf"):
            a.append([c.eid,pdf_url])
        else:
            z.append([c.eid,str])
    except:
        z.append([c.eid,str])


In [25]:
a

In [221]:
f

In [24]:
z

In [27]:
l = []

In [45]:
l.append(["2-s2.0-85047089156","https://doi.org/10.1109/ICTUS.2017.8286021"])

In [46]:
l

In [35]:
import subprocess

for c in a:
    # Nombre del archivo de salida (opcional, si wget no lo decide)
    filename = c[0],".pdf"
    # Ejecuta wget con la opción de content disposition
    subprocess.run(['wget', '--content-disposition', c[1], '-O', filename], check=True)

print(f"El archivo ha sido guardado como {filename}.")
